### Sequence-to-Sequence - Encoder-Decoder - English - French - Machine Translation

https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

we will use a dataset of pairs of English sentences and their French translation. \
We will implement a character-level sequence-to-sequence model, processing the input character-by-character and generating the output character-by-character.

Each word will be passed in form of Vectors . So we will convert each word to One Hot representation  .

Here's a summary of our process:

1) Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:
    encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a         one-hot vectorization of the English sentences.
    decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containg a one-hot     vectorization of the French sentences.
    decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the     same as decoder_input_data[:, t + 1, :].

2) Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data. Our      model uses teacher forcing.

3) Decode some sentences to check that the model is working (i.e. turn samples from encoder_input_data into corresponding          samples from decoder_target_data).

## Machine Learning Langauage Translation

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

batch_size=64
epochs=100
latent_dim = 256 # Latent Dimensionality of Encoding Space
num_samples = 10000 # No of samples to train on

## Path to data txt file on disk
data_path = 'fra.txt'

In [2]:
# Now we will read Input and Target data , W.r.t Input Sentences we are reading unique characters
input_texts =[]
target_texts = []
input_characters = set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
  lines = f.read().split('\n')

for line in lines[:min(num_samples,len(lines)-1)]:
  # English and French text is splitted at \t
  input_text,target_text,_ = line.split('\t')
  # We use tab as start sequence and '\n' as end sequence character for targets
  target_text = '\t' + target_text +'\n'
  input_texts.append(input_text)
  target_texts.append(target_text)

  ## Now we will read Input and Output sentences into character
  for char in input_text:
    ### Will only add unique characters in list
    if char not in input_characters:
      input_characters.add(char)
  
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [3]:
input_characters

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [4]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '’',
 '\u202f'}

In [5]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I fled.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',

In [6]:
target_texts

['\tVa !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tAttaque !\n',
 '\tAttaquez !\n',
 '\tSanté !\n',
 '\tÀ votre santé !\n',
 '\tMerci !\n',
 '\tTchin-tchin !\n',
 '\tLève-toi.\n',
 '\tLève-toi !\n',
 '\tVa, maintenant.\n',
 '\tAllez-y maintenant.\n',
 '\tVas-y maintenant.\n',
 "\tJ'ai pigé !\n",
 '\tCompris !\n',
 '\tPigé\u202f?\n',
 '\tCompris\u202f?\n',
 "\tT'as capté\u202f?\n",
 '\tMonte.\n',
 '\tMontez.\n',
 '\tSerre-moi dans tes bras !\n',
 '\tSerrez-moi dans vos bras !\n',
 '\tJe suis tombée.\n',
 '\tJe suis tombé.\n',
 "\tJ'ai fui.\n",

In [7]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of Sample : ',num_samples)
print('Number of unique input Tokens : ' , num_encoder_tokens)
print('Number of unique output Tokens : ' , num_decoder_tokens)
print('Max Sequence Length for inputs : ',max_encoder_seq_length)
print('Max Sequence Length for outputs : ',max_decoder_seq_length)

Number of Sample :  10000
Number of unique input Tokens :  71
Number of unique output Tokens :  94
Max Sequence Length for inputs :  15
Max Sequence Length for outputs :  59


### Now for Each character we will assign token

In [8]:
input_token_index = dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i,char in enumerate(target_characters)])

input_token_index , target_token_index

({' ': 0,
  '!': 1,
  '$': 2,
  '%': 3,
  '&': 4,
  "'": 5,
  ',': 6,
  '-': 7,
  '.': 8,
  '0': 9,
  '1': 10,
  '2': 11,
  '3': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '%': 5,
  '&': 6,
  "'": 7,
  '(': 8,
  ')': 9,
  ',': 10,
  '-': 11,
  '.': 12,
  '0': 13,
  '1': 14,
  '2': 15,
  '3': 16,
  '5': 17,
  '8': 18,
  '9': 19,
  ':': 20,
  '?'

## One Hot Representation  

Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:

1. encoder_input_data is a 3D array of shape (num_pairs - Total no of Sentences ) ,         max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences.

2. decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containg a one-hot vectorization of the French sentences.

3. decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :].

In [9]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32'
)
decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)
decoder_target_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32'
)

In [10]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I fled.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',

In [11]:
len(input_texts)

10000

In [12]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  ### Now input_texts is list of sentences , In sentence 1 - wherever that char is present replace it by one in encoder_input data
  for t,char in enumerate(input_text):
    ## encoder_input_data[length of input texts , max encoder_sequence kength , number of encoder tokens ]
    ## input_token_index is dictionary -- Will search the value of the char in that dictionary and replace that whole encoder input data value to 1
    encoder_input_data[i,t,input_token_index[char]] = 1
  encoder_input_data[i,t + 1: , input_token_index[' ']] = 1

  for t,char in enumerate(target_text):
    ## Decoder_target_data is ahead of decoder_input_data by one timestamp
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      ## Decoder target data is ahead by one timestamp and will not include start character
      decoder_target_data[i,t-1,target_token_index[char]] = 1

  decoder_input_data[i,t+1:,target_token_index[' ']] = 1
  decoder_target_data[i,t:,target_token_index[' ']] = 1


In [13]:
encoder_input_data[0].shape

(15, 71)

### Creating Model

LSTM LAYER 

#### Encoders

In [14]:
encoder_inputs = Input(shape=(None,num_encoder_tokens))  ## Encoder Input Shape -- No of unique English Characters
encoder = LSTM(latent_dim,return_state=True)  ### latent dim - Timestamps
encoder_outputs,state_h,state_c = encoder(encoder_inputs)

## We Discard Encoder_Outputs and only keep states
## state_h -- hidden state , state_c -- cell state -- Hidden cell output which is passed to next RNN Hidden state
encoder_states = [state_h,state_c]

#### Decoders

In [15]:
## Set up decoder , using encoder_states as initial state
decoder_inputs = Input(shape=(None,num_decoder_tokens))

## We setup decoder to return full output sequences 
## Also to return Internal states as well , We dont use return states in training model , but we will use them in inference

decoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True)

## Here now we are concern about output
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#### Model Creation using Encoder and Decoder

In [16]:
### We will use 'encoder_input_data' , 'decoder_input_data' , 'decoder_target_data'

model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

### Training

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 2s 17ms/step - loss: 1.1870 - accuracy: 0.7264 - val_loss: 1.0390 - val_accuracy: 0.7105
Epoch 2/100
125/125 [==============================] - 2s 13ms/step - loss: 0.8559 - accuracy: 0.7680 - val_loss: 0.8426 - val_accuracy: 0.7698
Epoch 3/100
125/125 [==============================] - 2s 13ms/step - loss: 0.6795 - accuracy: 0.8092 - val_loss: 0.7046 - val_accuracy: 0.7948
Epoch 4/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5883 - accuracy: 0.8299 - val_loss: 0.6496 - val_accuracy: 0.8102
Epoch 5/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5364 - accuracy: 0.8436 - val_loss: 0.6140 - val_accuracy: 0.8177
Epoch 6/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4989 - accuracy: 0.8534 - val_loss: 0.5793 - val_accuracy: 0.8293
Epoch 7/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4711 - accuracy: 0.8609 - val_loss: 0.5599 - val_accuracy:

### Try it on Testing Data

To decode a test sentence, we will repeatedly:

1. Encode the input sentence and retrieve the initial decoder state
2. Run one step of the decoder with this initial state and a "start of sequence" token as target. The output will be the next target character.
3. Append the target character predicted and repeat.

In [19]:
### Define Sampling Models

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


### Reverse Lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i,char) for char,i in input_token_index.items())

reverse_target_char_index = dict((i,char) for char,i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
  ## take one sentence from training set for trying out decoding

  input_seq = encoder_input_data[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print('---')
  print('Input Sentence : ' ,input_texts[seq_index])
  print('Decoded Sentence : ', decoded_sentence)

---
Input Sentence :  Go.
Decoded Sentence :  Va !

---
Input Sentence :  Hi.
Decoded Sentence :  Salut.

---
Input Sentence :  Hi.
Decoded Sentence :  Salut.

---
Input Sentence :  Run!
Decoded Sentence :  Courez !

---
Input Sentence :  Run!
Decoded Sentence :  Courez !

---
Input Sentence :  Who?
Decoded Sentence :  Qui ?

---
Input Sentence :  Wow!
Decoded Sentence :  Ça alors !

---
Input Sentence :  Fire!
Decoded Sentence :  Au feu !

---
Input Sentence :  Help!
Decoded Sentence :  À l'aide !

---
Input Sentence :  Jump.
Decoded Sentence :  Saute.

---
Input Sentence :  Stop!
Decoded Sentence :  Stop !

---
Input Sentence :  Stop!
Decoded Sentence :  Stop !

---
Input Sentence :  Stop!
Decoded Sentence :  Stop !

---
Input Sentence :  Wait!
Decoded Sentence :  Attends !

---
Input Sentence :  Wait!
Decoded Sentence :  Attends !

---
Input Sentence :  Go on.
Decoded Sentence :  Poursuis.

---
Input Sentence :  Go on.
Decoded Sentence :  Poursuis.

---
Input Sentence :  Go on.
Deco